In [1]:
import plotly.express as px


In [2]:
from api.api import API
api = API('')

In [3]:
df=api.reports('SELECT * FROM PIX_MOVEMENTS',False)

In [98]:
total=df.groupby('status')['pix_amount'].count()
status=df.groupby(['status','in_or_out'])['id'].count()
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Figure(data=[go.Pie(labels=total.index, values=total.values,pull=[0, 0.2])]),row=1,col=1)
# fig.add_trace(go.Figure(data=[
# go.Bar(name='Completed', x=status['completed'].index, y=status['completed'].values),
# go.Bar(name='Failed', x=status['failed'].index, y=status['failed'].values)]))

TypeError: append_trace() missing 2 required positional arguments: 'row' and 'col'

In [6]:
df.head()

,id,account_id,in_or_out,pix_amount,pix_requested_at,pix_completed_at,status
0,13629077-0946-8179-9680-000000000000,50928183-6645-3152-6400-000000000000,pix_out,1894.77,1579693633580,1.579694e+12,completed
1,22467941-1802-2659-0720-000000000000,50928183-6645-3152-6400-000000000000,pix_out,419.79,1587309244550,1.587309e+12,completed
2,16569602-6045-6373-7600-000000000000,50928183-6645-3152-6400-000000000000,pix_in,943.61,1609306218320,1.609306e+12,completed
3,91954593-2589-0464-0000-000000000000,50928183-6645-3152-6400-000000000000,pix_in,124.66,1586015734430,1.586016e+12,completed
4,20012121-4224-0523-0080-000000000000,50928183-6645-3152-6400-000000000000,pix_out,775.96,1590974667000,1.590975e+12,completed


In [7]:
a=df.groupby(['status','in_or_out'])['id'].count()

In [35]:
max(a.values)

118666

In [36]:
from plotly.subplots import make_subplots


In [95]:
fig=go.Figure(data=[
    go.Bar(name='Completed', x=a['completed'].index, y=a['completed'].values),
    go.Bar(name='Failed', x=a['failed'].index, y=a['failed'].values)
])
fig.add_hline(y=max(a.values),line_dash="dash",opacity=0.7,x0=0.02,x1=0.98)
fig.add_hline(y=min(a.values),line_dash="dash",opacity=0.7,x0=0.02,x1=0.98)


                  

In [28]:
a['completed']

in_or_out
pix_in     118617
pix_out    118666
Name: id, dtype: int64

In [12]:
a.index

MultiIndex([('completed',  'pix_in'),
            ('completed', 'pix_out'),
            (   'failed',  'pix_in'),
            (   'failed', 'pix_out')],
           names=['status', 'in_or_out'])

In [18]:
import pandas as pd

In [21]:
f=pd.DataFrame(a)

In [27]:
pd.DataFrame(a)

id
status    in_or_out        
completed pix_in     118617
          pix_out    118666
failed    pix_in       6316
          pix_out      6283

In [22]:
f.unstack()

id        
in_or_out  pix_in pix_out
status                   
completed  118617  118666
failed       6316    6283

In [23]:
fig = go.Figure(data=f.unstack()).update_layout()
fig.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [               id        
in_or_out  pix_in pix_out
status                   
completed  118617  118666
failed       6316    6283]

    The 'data' property is a tuple of trace instances
    that may be specified as:
      - A list or tuple of trace instances
        (e.g. [Scatter(...), Bar(...)])
      - A single trace instance
        (e.g. Scatter(...), Bar(...), etc.)
      - A list or tuple of dicts of string/value properties where:
        - The 'type' property specifies the trace type
            One of: ['bar', 'barpolar', 'box', 'candlestick',
                     'carpet', 'choropleth', 'choroplethmapbox',
                     'cone', 'contour', 'contourcarpet',
                     'densitymapbox', 'funnel', 'funnelarea',
                     'heatmap', 'heatmapgl', 'histogram',
                     'histogram2d', 'histogram2dcontour', 'icicle',
                     'i

In [ ]:
df['state']

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
a

In [ ]:
a.index

In [14]:
import plotly.graph_objects as go


In [ ]:
fig=go.Figure(data=[go.Pie(labels=a.index, values=a.values,pull=[0, 0.2])])
fig.update_layout(title_text='PIX Transaction status 2020')